# Setup

In [2]:
import os
import logging as log

from lisa.utils import setup_logging
setup_logging(level=log.ERROR)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import polars as pl
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate
from pathlib import Path

from holoviews.operation.datashader import datashade, rasterize
from holoviews.operation import decimate

from wp.notebook import WorkloadNotebookAnalysis, setup_notebook

setup_notebook()

## Runs

In [13]:
jet = WorkloadNotebookAnalysis('/home/kajetan/uofg/ml-idle/workloads', [
    'jetnews_menu_idle_10_1412',
    'jetnews_teo_deltatrace_10_1703',
    'jetnews_mlidle_7f_8d_10_2003',
    'jetnews_mlidle_7f_10_2003',
])

jet.show()

,metric,value,unit,iteration,workload,id,wa_path,kernel,tag
0,seconds,79.3810,,1,jetnews,wk1,jetnews_menu_idle_10_1412,6.3.0-mainline-g7c68bfdc80c5-dirty,menu_idle
1,fps,43.2874,fps,1,jetnews,wk1,jetnews_menu_idle_10_1412,6.3.0-mainline-g7c68bfdc80c5-dirty,menu_idle
2,total_frames,3917.0000,frames,1,jetnews,wk1,jetnews_menu_idle_10_1412,6.3.0-mainline-g7c68bfdc80c5-dirty,menu_idle
3,janks,21.0000,count,1,jetnews,wk1,jetnews_menu_idle_10_1412,6.3.0-mainline-g7c68bfdc80c5-dirty,menu_idle
4,janks_pc,0.4000,percent,1,jetnews,wk1,jetnews_menu_idle_10_1412,6.3.0-mainline-g7c68bfdc80c5-dirty,menu_idle
...,...,...,...,...,...,...,...,...,...
795,frame_deadline_missed_(legacy),55.0000,count,10,jetnews,wk1,jetnews_mlidle_7f_10_2003,6.3.0-mainline-g664fb1e13797-dirty,mlidle_7f
796,render_time_50th gpu_ptile,1.0000,milliseconds,10,jetnews,wk1,jetnews_mlidle_7f_10_2003,6.3.0-mainline-g664fb1e13797-dirty,mlidle_7f
797,render_time_90th gpu_ptile,2.0000,milliseconds,10,jetnews,wk1,jetnews_mlidle_7f_10_2003,6.3.0-mainline-g664fb1e13797-dirty,mlidle_7f
798,render_time_95th gpu_ptile,4.0000,milliseconds,10,jetnews,wk1,jetnews_mlidle_7f_10_2003,6.3.0-mainline-g664fb1e13797-dirty,mlidle_7f


benchmark_dirs: ['jetnews_menu_idle_10_1412', 'jetnews_teo_deltatrace_10_1703', 'jetnews_mlidle_7f_8d_10_2003', 'jetnews_mlidle_7f_10_2003']
tags: ['menu_idle', 'teo_deltatrace', 'mlidle_7f_8d', 'mlidle_7f']
kernels: ['6.3.0-mainline-g7c68bfdc80c5-dirty', '6.3.0-mainline-g664fb1e13797-dirty', '6.3.0-mainline-g664fb1e13797-dirty', '6.3.0-mainline-g664fb1e13797-dirty']


# Benchmark scores

In [4]:
jet.plot.results_line(['fps', 'janks_pc'])

:NdLayout   [metric]
   :NdOverlay   [tag]
      :Curve   [iteration]   (value)

In [14]:
jet.plot.results_bar(['fps', 'janks_pc'])

+----------+----------------+-------+-----------+
|  metric  |      tag       | value | perc_diff |
+----------+----------------+-------+-----------+
|   fps    |   menu_idle    | 43.2  |   0.0%    |
|   fps    | teo_deltatrace | 42.5  |  -1.58%   |
|   fps    |  mlidle_7f_8d  | 42.8  |  -1.01%   |
|   fps    |   mlidle_7f    | 42.9  |  -0.64%   |
| janks_pc |   menu_idle    |  0.4  |   0.0%    |
| janks_pc | teo_deltatrace |  0.4  |   5.24%   |
| janks_pc |  mlidle_7f_8d  |  0.5  |  25.85%   |
| janks_pc |   mlidle_7f    |  0.6  |  33.85%   |
+----------+----------------+-------+-----------+


## Line plot

In [ ]:
jank.plot.overutilized_line()

# Perf

## Line plot

In [ ]:
jank.plot.perf_line()

## Bar plot

In [ ]:
jank.plot.perf_bar()

# Idle residency

## Bar plot

In [23]:
fig = px.bar(
    jet.analysis['idle_residency'], x='idle_state', y='time', color='tag',
    facet_col='cluster', barmode='group', text=jet.analysis['idle_residency']['time'],
    width=600, height=500, title='Idle state residencies'
)
fig.update_traces(textposition='outside')
fig.show(renderer='iframe')

In [18]:
jet.plot.idle_residency_bar()

+----------------+---------+------------+-------+
|      tag       | cluster | idle_state | time  |
+----------------+---------+------------+-------+
|   menu_idle    | little  |    -1.0    | 44.39 |
|   menu_idle    | little  |    0.0     | 14.23 |
|   menu_idle    | little  |    1.0     | 23.78 |
|   menu_idle    |   mid   |    -1.0    | 23.62 |
|   menu_idle    |   mid   |    0.0     | 12.78 |
|   menu_idle    |   mid   |    1.0     | 46.0  |
|   menu_idle    |   big   |    -1.0    | 16.64 |
|   menu_idle    |   big   |    0.0     | 20.72 |
|   menu_idle    |   big   |    1.0     | 45.04 |
| teo_deltatrace | little  |    -1.0    | 45.31 |
| teo_deltatrace | little  |    0.0     | 24.27 |
| teo_deltatrace | little  |    1.0     | 13.39 |
| teo_deltatrace |   mid   |    -1.0    | 21.72 |
| teo_deltatrace |   mid   |    0.0     | 38.31 |
| teo_deltatrace |   mid   |    1.0     | 22.94 |
| teo_deltatrace |   big   |    -1.0    | 16.27 |
| teo_deltatrace |   big   |    0.0     | 38.48 |


# Idle misses

In [15]:
def workload_idle_baseline_accuracy(path):
    print('Baseline for', path)
    cpu_idle = pl.read_parquet(path / 'analysis' / 'cpu_idle.pqt')
    wakeups = cpu_idle.filter(pl.col('state') == -1)
    cpu_idle_miss = pl.read_parquet(path / 'analysis' / 'cpu_idle_miss.pqt')
    
    print('accuracy', 1 - cpu_idle_miss.height / wakeups.height)
    print('misses', cpu_idle_miss.height / wakeups.height)
    print('too deep', cpu_idle_miss.filter(pl.col('state') == 1).height / wakeups.height, 'too shallow', cpu_idle_miss.filter(pl.col('state') == 0).height / wakeups.height)
    print()

    return (wakeups.height, cpu_idle_miss.height)

for out in jet.wa_outputs:
    path = Path(out.path)
    workload_idle_baseline_accuracy(path)

Baseline for /home/kajetan/uofg/ml-idle/workloads/jetnews_menu_idle_10_1412
accuracy 0.6962507146510999
misses 0.30374928534890017
too deep 0.2697473562088553 too shallow 0.03400192914004487

Baseline for /home/kajetan/uofg/ml-idle/workloads/jetnews_teo_deltatrace_10_1703
accuracy 0.7945433631975234
misses 0.20545663680247658
too deep 0.029864873284886007 too shallow 0.17559176351759057

Baseline for /home/kajetan/uofg/ml-idle/workloads/jetnews_mlidle_7f_8d_10_2003
accuracy 0.7970261919434665
misses 0.20297380805653348
too deep 0.03587078669654833 too shallow 0.16710302135998517

Baseline for /home/kajetan/uofg/ml-idle/workloads/jetnews_mlidle_7f_10_2003
accuracy 0.7846725589192568
misses 0.21532744108074323
too deep 0.0353003379392982 too shallow 0.18002710314144502



## Bar plot

In [8]:
jet.plot.idle_miss_bar()

+----------------+-------------+--------------------+
|      tag       |    type     |     count_perc     |
+----------------+-------------+--------------------+
|   menu_idle    |  too deep   |       26.975       |
|   menu_idle    | too shallow | 3.4000000000000004 |
|   mlidle_7f    |  too deep   | 3.5300000000000002 |
|   mlidle_7f    | too shallow |       18.003       |
| teo_deltatrace |  too deep   |       2.987        |
| teo_deltatrace | too shallow | 17.558999999999997 |
+----------------+-------------+--------------------+


# Power usage

## Line plot

In [9]:
jet.plot.power_meter_line()

## Bar plot

In [16]:
jet.plot.power_meter_bar()

+------------+--------+----------------+-------+-----------+
|  channel   | metric |      tag       | value | perc_diff |
+------------+--------+----------------+-------+-----------+
|    CPU     | gmean  |   menu_idle    | 147.4 |   0.0%    |
|  CPU-Big   | gmean  |   menu_idle    | 40.7  |   0.0%    |
| CPU-Little | gmean  |   menu_idle    | 80.4  |   0.0%    |
|  CPU-Mid   | gmean  |   menu_idle    | 26.3  |   0.0%    |
|    GPU     | gmean  |   menu_idle    | 25.3  |   0.0%    |
|   Total    | gmean  |   menu_idle    | 172.8 |   0.0%    |
|    CPU     | gmean  | teo_deltatrace | 142.3 |  -3.47%   |
|  CPU-Big   | gmean  | teo_deltatrace | 36.5  |  -10.28%  |
| CPU-Little | gmean  | teo_deltatrace | 77.8  |  -3.23%   |
|  CPU-Mid   | gmean  | teo_deltatrace | 27.9  |   6.15%   |
|    GPU     | gmean  | teo_deltatrace | 24.9  |  -1.44%   |
|   Total    | gmean  | teo_deltatrace | 167.3 |  -3.14%   |
|    CPU     | gmean  |  mlidle_7f_8d  | 145.1 |  -1.61%   |
|  CPU-Big   | gmean  | 

# Energy estimate

## Line plot

In [ ]:
jank.plot.energy_estimate_line()

## Bar plot

In [ ]:
jank.plot.energy_estimate_bar()

# Thermal

## Line plot

In [ ]:
jank.plot.thermal_line()

## Bar plot

In [ ]:
jank.plot.thermal_bar()

# Frequency

## Line plot

In [ ]:
jank.plot.frequency_line()

## Bar plot

In [ ]:
jank.plot.frequency_bar()

# CFS signals

## Line plot

In [ ]:
jank.plot.sched_pelt_cfs_line()

## Bar plot

In [ ]:
jank.plot.sched_pelt_cfs_bar()

# Wakeup latency - tasks

## Line plot

In [ ]:
jank.plot.wakeup_latency_line()

## Bar plot

In [ ]:
jank.plot.wakeup_latency_bar()

## Quantiles

In [ ]:
jank.plot.wakeup_latency_quantiles_bar()

# Wakeup latency - cgroups

## Line plot

In [ ]:
jank.plot.wakeup_latency_cgroup_line()

## Bar plot

In [ ]:
jank.plot.wakeup_latency_cgroup_bar()

## Quantiles

In [ ]:
jank.plot.wakeup_latency_cgroup_quantiles_bar()

# CPU residency - tasks

## Clusters - Line plot

In [ ]:
jank.plot.tasks_cpu_residency_cluster_line()

## Clusters - Bar plot

In [ ]:
jank.plot.tasks_cpu_residency_cluster_bar()

## Clusters - Per-task bar plot

In [ ]:
jank.plot.tasks_cpu_residency_per_task_bar()

## CPUs - Bar plot

In [ ]:
jank.plot.tasks_cpu_residency_cpu_bar()

# CPU residency - cgroups

## Clusters - Bar plot

In [ ]:
jank.plot.cgroup_cpu_residency_cluster_bar()

## CPUs - Bar plot

In [ ]:
jank.plot.cgroup_cpu_residency_cpu_bar()

# Summary - TLDR

In [12]:
jet.plot.summary()

Jetnews
+-----------------+--------------+----------------+----------------+
|     metric      |  menu_idle   | teo_deltatrace |   mlidle_7f    |
+-----------------+--------------+----------------+----------------+
|       fps       | 43.2 (0.0%)  | 42.5 (-1.58%)  | 42.9 (-0.64%)  |
|    janks_pc     |  0.4 (0.0%)  |  0.4 (5.24%)   |  0.6 (33.85%)  |
| CPU_total_power | 147.4 (0.0%) | 142.3 (-3.47%) | 144.3 (-2.12%) |
+-----------------+--------------+----------------+----------------+


tag,metric,menu_idle,teo_deltatrace,mlidle_7f
0,fps,43.2 (0.0%),42.5 (-1.58%),42.9 (-0.64%)
1,janks_pc,0.4 (0.0%),0.4 (5.24%),0.6 (33.85%)
2,CPU_total_power,147.4 (0.0%),142.3 (-3.47%),144.3 (-2.12%)


## Save image plots

In [ ]:
jank.save_image_plots('/home/kajpuc01/temp/jank_vh_impact')